In [14]:
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# URLs for datasets
datasets = {
    "SO2TONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/SO2TONS_dataset.csv",
    "NOXTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/NOXTONS_dataset.csv",
    "COTONS": "https://raw.githubusercontent.com/apownukepcc/ForecastingDailyEmissions/refs/heads/main/COTONS_dataset.csv"
}

# Define the days for prediction
specific_dates = {
    "Peak Season": pd.Timestamp("2022-07-15"),
    "Off-Peak Season 1": pd.Timestamp("2022-02-15"),
    "Off-Peak Season 2": pd.Timestamp("2022-10-15"),
}

# Define lakes (sources)
sources = ["LAKE-1", "LAKE-2", "LAKE-3", "LAKE-4"]

# Initialize a dictionary to store models, predictions, and efficiency
predictions = {}
efficiency = {}

# Helper function to determine season
def get_season(date):
    if date.month >= 5 and date.month <= 8:
        return 'Peak Season'
    else:
        return 'Off-Peak Season'

# Corrected helper function to find the closest available date within the same season
def find_closest_date(date, available_dates):
    season = get_season(date)
    same_season_dates = available_dates[available_dates.apply(get_season) == season]
    if same_season_dates.empty:
        return None
    time_deltas = (same_season_dates - date).abs()
    closest_index = time_deltas.idxmin()
    closest_date = same_season_dates.loc[closest_index]
    return closest_date

# Loop through each dataset (SO2TONS, NOXTONS, COTONS)
for parameter, url in datasets.items():
    # Load the dataset
    data = pd.read_csv(url)

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Separate data by source
    for source in sources:
        source_data = data[data['Source'] == source]

        # Check if the source data has enough rows
        if source_data.empty or len(source_data) < 10:
            print(f"Not enough data for {parameter} at {source}. Skipping...")
            continue

        # Define predictors and target
        predictors = ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']
        target = 'Emissions_Load'

        # Drop rows with missing values
        source_data = source_data.dropna(subset=predictors + [target])

        # Ensure that the predictors are in the DataFrame
        missing_predictors = [col for col in predictors if col not in source_data.columns]
        if missing_predictors:
            print(f"Missing predictors {missing_predictors} in data for {parameter} at {source}. Skipping...")
            continue

        # Split into features (X) and target (y)
        X = source_data[predictors]
        y = source_data[target]

        # Ensure sufficient data for training
        if len(X) < 10:
            print(f"Not enough data after dropping missing values for {parameter} at {source}. Skipping...")
            continue
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Hyperparameter tuning using GridSearchCV
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2],
            'max_features': ['auto', 'sqrt']
        }
        rf = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_rf = grid_search.best_estimator_

        # Evaluate the model
        y_pred = best_rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"Model for {parameter} at {source} (Random Forest):")
        print(f"  Best Parameters: {grid_search.best_params_}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R²: {r2:.4f}")

        # Feature importance
        importances = best_rf.feature_importances_
        feature_importance = pd.Series(importances, index=predictors).sort_values(ascending=False)
        print(f"  Feature Importances:\n{feature_importance}\n")

        # Save predictions for specific days
        predictions[(parameter, source)] = {}
        for season_label, specific_date in specific_dates.items():
            # Determine if the date matches the season
            date_season = get_season(specific_date)
            if (season_label == "Peak Season" and date_season != "Peak Season") or \
               (season_label.startswith("Off-Peak") and date_season != "Off-Peak Season"):
                # Skip dates that don't match the intended season
                predictions[(parameter, source)][season_label] = None
                continue

            # Get available dates
            available_dates = source_data['date']

            # Check if exact date exists, otherwise find the closest date within the same season
            if specific_date not in available_dates.values:
                closest_date = find_closest_date(specific_date, available_dates)
                if closest_date is None:
                    print(f"No available dates in the same season for {parameter} at {source}.")
                    predictions[(parameter, source)][season_label] = None
                    continue
                print(f"Closest date to {specific_date.date()} for {parameter} at {source} in the same season: {closest_date.date()}")
                specific_date = closest_date

            # Filter data for the specific (or closest) date
            day_data = source_data[source_data['date'] == specific_date]
            if not day_data.empty:
                # Re-index day_data to ensure iloc works correctly
                day_data = day_data.reset_index(drop=True)

                # Ensure that day_data has at least one row
                if len(day_data) > 0:
                    # Ensure that predictors and target are in the day_data
                    if all(col in day_data.columns for col in predictors + [target]):
                        specific_features = day_data[predictors].iloc[0]
                        specific_actual = day_data[target].iloc[0]

                        if not specific_features.isnull().any() and not pd.isnull(specific_actual):
                            # Predict emissions/load for the specific day
                            specific_prediction = best_rf.predict([specific_features])[0]

                            # Save the prediction and actual value for verification
                            predictions[(parameter, source)][season_label] = {
                                "features": specific_features,
                                "actual": specific_actual,
                                "predicted": specific_prediction,
                                "efficiency": specific_actual  # Efficiency as actual emissions
                            }
                        else:
                            print(f"Missing data in features or target for {parameter} at {source} on {specific_date.date()}.")
                            predictions[(parameter, source)][season_label] = None
                    else:
                        print(f"Missing predictors or target in data for {parameter} at {source} on {specific_date.date()}.")
                        predictions[(parameter, source)][season_label] = None
                else:
                    print(f"No valid data available for {parameter} at {source} on {specific_date.date()}.")
                    predictions[(parameter, source)][season_label] = None
            else:
                print(f"No data available for {parameter} at {source} on {specific_date.date()}.")
                predictions[(parameter, source)][season_label] = None

# Calculate relative efficiency using actual emissions when available
for source in sources:
    total_emissions = 0
    count = 0
    for (parameter, unit), season_results in predictions.items():
        if unit == source:
            for season, result in season_results.items():
                if result and "actual" in result:
                    total_emissions += result["actual"]
                    count += 1
    efficiency[source] = total_emissions / count if count > 0 else None

# Display all predictions
print("\nFinal Predictions:")
for key, season_results in predictions.items():
    parameter, source = key
    print(f"\n{parameter} at {source}:")
    for season, result in season_results.items():
        if result:
            print(f"  {season}:")
            print(f"    Features: {result['features'].to_dict()}")
            print(f"    Actual Emissions_Load: {result['actual']:.4f}")
            print(f"    Predicted Emissions_Load: {result['predicted']:.4f}")
            print(f"    Efficiency (Actual Emissions_Load): {result['efficiency']:.4f}")
        else:
            print(f"  {season}: No data available.")

# Print efficiency results
print("\nEfficiency Analysis:")
most_efficient_unit = None
min_emission = float('inf')
for source, eff in efficiency.items():
    if eff is not None and eff < min_emission:
        min_emission = eff
        most_efficient_unit = source
    if eff is not None:
        print(f"{source}: Average Actual Emissions = {eff:.6f}")
    else:
        print(f"{source}: No efficiency data available.")
if most_efficient_unit:
    print(f"\nThe most efficient unit is: {most_efficient_unit}")
else:
    print("\nNo efficiency data available to determine the most efficient unit.")




# Initialize a nested dictionary to store efficiency per parameter
efficiency_per_parameter = {param: {} for param in datasets.keys()}

# Calculate efficiency per parameter
for parameter in datasets.keys():
    for source in sources:
        total_emissions = 0
        count = 0
        for (param, unit), season_results in predictions.items():
            if param == parameter and unit == source:
                for season, result in season_results.items():
                    if result and "actual" in result:
                        total_emissions += result["actual"]
                        count += 1
        avg_emission = total_emissions / count if count > 0 else None
        efficiency_per_parameter[parameter][source] = avg_emission

# Display efficiency per parameter
for parameter, unit_efficiencies in efficiency_per_parameter.items():
    print(f"\nEfficiency Analysis for {parameter}:")
    for source, eff in unit_efficiencies.items():
        if eff is not None:
            print(f"{source}: Average Actual Emissions = {eff:.6f}")
        else:
            print(f"{source}: No efficiency data available.")

    # Determine the most efficient unit for this parameter
    valid_efficiencies = {k: v for k, v in unit_efficiencies.items() if v is not None}
    if valid_efficiencies:
        most_efficient_unit = min(valid_efficiencies, key=valid_efficiencies.get)
        print(f"The most efficient unit for {parameter} is: {most_efficient_unit}")
    else:
        print(f"No efficiency data available to determine the most efficient unit for {parameter}.")


Model for SO2TONS at LAKE-1 (Random Forest):
  Best Parameters: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
  RMSE: 0.0000
  R²: -0.0278
  Feature Importances:
tmax    0.198622
wdir    0.164684
tavg    0.162057
tmin    0.156433
pres    0.143358
wspd    0.141720
prcp    0.033126
snow    0.000000
dtype: float64

Closest date to 2022-10-15 for SO2TONS at LAKE-1 in the same season: 2022-10-14
Model for SO2TONS at LAKE-2 (Random Forest):
  Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
  RMSE: 0.0000
  R²: -0.0306
  Feature Importances:
wspd    0.212421
pres    0.173068
tmin    0.165407
tmax    0.148712
tavg    0.148074
wdir    0.134752
prcp    0.017516
snow    0.000049
dtype: float64

Closest date to 2022-02-15 for SO2TONS at LAKE-2 in the same season: 2022-02-07
Model for SO2TONS at LAKE-3 (Random Forest):
  Best Parameters: {'max_depth': 10, 